# ハイパースペクトラルセンサーデータの可視化
- [このページのJupyter Notebookファイル](https://amano-takahisa.github.io/rs_with_python/handson/plot_hyperspectral.ipynb)
- <a href="https://colab.research.google.com/github/amano-takahisa/rs_with_python/blob/master/docs/handson/plot_hyperspectral.ipynb" target="_blank">google colaboratoryで開く</a>

ハイパースペクトラルセンサーデータを用いて、Pythonでの基本的なデータ処理について学びます。

センサーデータのCSVから、以下のようなグラフと各測定でのNDVIの値を算出します。

![out](https://github.com/amano-takahisa/rs_with_python/raw/master/source/handson/out/spectral.png)

利用するデータは、[こちら](https://amano-takahisa.github.io/rs_with_python/handson/data/spectrum2019.csv)。ダウンロードの上、ファイルマネージャにて`data`フォルダを作成し、その中に保存してください。

## CSVファイルの読み込み
CSVやエクセルなどに保存された表敬式のデータを扱うには、**Pandas**というライブラリを利用します。  
[Pandasの公式ドキュメントはこちら](https://pandas.pydata.org/pandas-docs/stable/index.html)

Pandasでは表形式のデータを**DataFrame**と呼ばれる形で保持し、必要な処理を行っていきます。

まずはPandasを用いてデータを読み込んでみましょう。

In [ ]:
import pandas as pd  # pandasをpdという別名で呼び出す

# csvファイルを読み込み、dfという変数に代入する


各業の意味はコメントに記載のとおりです。
Pandasライブラリには、今回利用したCSVファイルを読み込むための`read_csv()`メソッド(関数)の他にも、エクセルファイルを読み込むための`read_excel()`、クリップボードからデータを読み込む`read_clipboard()`などのメソッドが用意されています。詳しくは[Pandas API reference](https://pandas.pydata.org/pandas-docs/stable/reference/io.html)で確認してください。

データは、dfに`DataFrame`というクラスで代入されています。

In [ ]:
# オブジェクトタイプの確認


また、表示されるテーブルの最後に、`5048 rows × 5 columns`と表示があり、CSVデータは5,048行、5列のデータであることがわかります。

このCSVデータは、列ごとに各観測が保存され、46行目までにメタデータ、47行目以降が各観測の波長ごとの放射照度(分光放射照度)の値が保存されています。また、2,548行目から別の測定が保存されています。

今回はメタデータ部分を飛ばして、かつ2,547行目までの放射照度の値のみを読み込んだデータフレームを作成します。

`skiprows=n`オプションを与えることで、ファイルのはじめのn行を飛ばして読み込むことができました。その他のオプションは[公式ドキュメント](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html)を参照してください。


## データのクリーニング

読み込んだデータをプロットする前に、データのクリーニングとして以下を実施します。
- カラム名の変更
- 欠損値の設定

### カラム名の変更
まずはカラムの名前が長すぎて扱いづらいので変更します。

### データ型の変更
次に、データ型の変更です。

データフレームに格納されるデータは、同じ列の値がすべて同じ型のであることが必要です。今回は数値としてデータを扱いたいため、すべての列を浮動小数点のデータとして読み込みます。
まず、以下のコードで各列がどのような型で読み込まれているかを確認します。



`read_csv()`関数は、各列に最適なデータ型を認識して読み込みます。読み込むデータに"NA"という文字列や値が無いセルは欠損値と認識し、浮動小数点のNaNという値に変換されます。
WaveLength、obs3、obs4の列はすべて数値(float64)として読み込むことができました。

obs1、obs2の列は、CSVファイルには数値の代わりに`nodata`の文字列が保存されている箇所があり、すべての値を数値に変換することができませんでした。
そのため、データフレームの各列は**object**の型で保持されています。

以下で、この数値に変換できなかった値を強制的にNaNへ変換し、すべての列を浮動小数点の型とすることができます。

すべてのカラムが浮動小数点に変換されました。  先程`nodata`と表示されていた箇所も、`NaN`と表示されています。

### 基本統計量の確認
データフレームに保存されているデータの概要は、`describe()`メソッドで確認することができます。

DataFrameクラスが持つ`count()`や`mean()`、`std()`などのメソッドを用いると、上記の基本統計量を個別に算出することができます。

`mean()`では、各カラムの平均値が、`std()`では各カラムの標準偏差の値が算出されていることが確認できます。

### 欠損値の設定
`irrad`のWaveLengthが400から403の値のデータを確認してみます。

いくつかの分光放射照度の値が負になっていることが確認できますが、分光放射照度は負の値を取らないため、これらは測定ノイズであると考えられます。

今回は負の値になった箇所を、欠損値であることを示す`NaN`で埋めていきます。

In [ ]:
# WaveLength以外のカラム名を設定

# target_columnsの値が負のセルをマスク

# 上記のセルと同じ箇所を表示

再度、データフレームの基本統計量を確認します。

最小値が0になり、その他の統計量も若干変化したことが確認できます。

## グラフへのプロット

上で作成したデータフレームをグラフにプロットしましょう。DataFrameクラスにある[plot()](https://pandas.pydata.org/pandas-docs/version/0.23.4/generated/pandas.DataFrame.plot.html)メソッドでプロットしてみます。

引数xにx軸としたいカラムの名前を指定します。

## NDVIの計算
NDVIは植生の状態をよく反映する指標で、以下の式で定義されます。
$$
    NDVI = \frac{近赤外線(NIR)の反射率 - 赤色(Red)の反射率}{近赤外線(NIR)の反射率 + 赤色(Red)の反射率}
$$

マルチスペクトルメータの場合、近赤外の反射率に相当するものは、上記の図の灰色の範囲の平均値、同様に赤色の反射率は赤色の範囲の平均値となります。今回のようなノイズが多いデータの場合、平均値より中央値を用いる方が良いでしょう。

ここからNDVIを算出するため、以下の手順を行います。
- 各バンドにおける中央値の計算
- NDVIの計算

### バンドごとの中央値の計算
作成したデータフレームから、特定のWave Lengthの範囲における各バンドの値の中央値は、以下の手順で求めることができる。
1. Wabe Lengthが、赤色の範囲である行のみをデータフレームから抜き出す。
2. 抜き出してできる新たなデータフレームの列ごとの平均値(中央値)を求める。
3. 上記1,2をNIRについても行う。

#### 行の抽出
データフレームから行番号を指定して特定の行を抜き出すには、`df[0:5]`のような形で行の範囲を指定します。

また、`0:5`の代わりに、`[True, False, False, ...]`のようなブール値の入ったリストを与えると、`True`の行を抽出することができます。(`[]`が入れ子になっていることに注意。)

更に、以下のようにカラムを抽出し条件式に当てはめると、N番目の行が条件を満たす場合`True`、満たさない場合`False`が入った**Series**を作成することができ、このSeriesも上記のリスト同様に行の抽出に使うことができます。

In [ ]:
# データフレームからのカラムの抽出

In [ ]:
# Bool値の入ったSeriesの作成

このBool値が入ったSeriesは、`&`や`|`で結合することで、複数の条件を満たす新たなBool値のSeriesを作成することができます。

以上を組み合わせて、赤色の波長範囲の分光放射照度のみのデータフレームは以下のように作成できます。

In [ ]:
# Bool値のSeriesを作る
# Seriesでデータフレームを抽出する

NIRについても同様にデータフレームを作成できます。

#### 中央値の計算
基本統計量の確認の際に示した`mean()`と同様に、DataFrameが持つ`median()`メソッドにより各カラムの中央値が計算できます。

### NDVIの計算
作成したSeriesからNDVIの計算に不要な`WaveLength`の値を抜いておきます。

Seriesは要素数が同じ場合、四則演算は各要素ごとに行われます。これを分配法則といいます。このため、各obsにおけるNDVIは定義式のようにSeriesを計算することにより求めることができます。

以上でNDVIの算出ができました。